In [1]:
import datetime

import numpy as np
import pandas as pd

import vectorbt as vbt

In [2]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=7)

interval = "1m"

In [3]:
symbols = [
    "BTC-USD",
    "ETH-USD",
    "ETH-BTC",
]

In [4]:
price = vbt.YFData.download(
    symbols = symbols,
    missing_index='drop',
    interval=interval,
    start = start_time,
    end = end_time
).get('Close')

/home/oskar/anaconda3/envs/vectorbt/lib/python3.10/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [5]:
# price.to_pickle('price.pkl')

In [6]:
# this is a wierd strategy, but it shows what can be done with custom indicators
def custom_indicator(close, rsi_window = 14, ma_window = 50, entry_rsi = 45, exit_rsi = 55):
    assert isinstance(close, pd.DataFrame) or isinstance(close, pd.Series)

    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    
    rsi, _ = rsi.align(
        close,
        broadcast_axis=0,
        method="ffill",
        join='right'
    )

    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()

    # when rsi is above 70 then -1 (sell)
    trend = np.where(rsi > exit_rsi, -1, 0)
    # when rsi is below 30 and price is below moving average then 1 (buy)
    trend = np.where((rsi < entry_rsi) & (close < ma), 1, trend)

    return trend

In [7]:
indicator = vbt.IndicatorFactory(
    class_name = 'Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry_rsi', 'exit_rsi'],
    output_names=['value']
).from_apply_func(
    custom_indicator,
    rsi_window=14,
    ma_window=50,
    entry_rsi=45,
    exit_rsi=55,
    keep_pd=True # close will be passed as pandas series instead of numpy array
)

In [8]:
rsi_windows = np.arange(start=10, stop=200, step=15, dtype=int)
ma_windows = np.arange(start=10, stop=200, step=15, dtype=int)
rsi_entries = np.arange(start=10, stop=40, step=10, dtype=float)
rsi_exits = np.arange(start=60, stop=90, step=10, dtype=float)

res = indicator.run(
    price,
    rsi_window=rsi_windows,
    ma_window=ma_windows,
    entry_rsi=rsi_entries,
    exit_rsi=rsi_exits,
    param_product=True
)

In [9]:
# res.value

In [10]:
entries = res.value == 1
exits = res.value == -1
pf = vbt.Portfolio.from_signals(price, entries, exits, fees=0.001, freq=interval)

In [11]:
returns = pf.total_return()

best_params = returns.idxmax()
worst_params = returns.idxmin()

print(f'Best params: {best_params} with return {returns.max()}')
print(f'Worst params: {worst_params} with return {returns.min()}')

Best params: (55, 10, 30.0, 80.0, 'ETH-USD') with return 0.04488676006153014
Worst params: (10, 55, 30.0, 60.0, 'ETH-USD') with return -0.1454939088597142


In [12]:
# returns.groupby(['comb_entry_rsi', 'comb_exit_rsi']).mean()
returns

comb_rsi_window  comb_ma_window  comb_entry_rsi  comb_exit_rsi  symbol 
10               10              10.0            60.0           BTC-USD   -0.029932
                                                                ETH-USD   -0.036617
                                                                ETH-BTC   -0.012751
                                                 70.0           BTC-USD   -0.025832
                                                                ETH-USD   -0.029615
                                                                             ...   
190              190             30.0            70.0           ETH-USD    0.000000
                                                                ETH-BTC    0.000000
                                                 80.0           BTC-USD    0.000000
                                                                ETH-USD    0.000000
                                                                ETH-BTC    0.000000
Name

In [13]:
# filter returns to only keep one symbol
# returns[returns.index.isin(['BTC-USD'], level='symbol')]

In [14]:
fig = returns.vbt.heatmap(
    x_level='comb_rsi_window',
    y_level='comb_ma_window', 
    slider_level='symbol', symmetric=False,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

In [15]:
fig = returns.vbt.volume(
    x_level='comb_rsi_window',
    y_level='comb_ma_window', 
    z_level='comb_entry_rsi',
    slider_level='symbol',
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

In [16]:
pf.plot(column=worst_params, show_titles=True, title=f'Portfolio {worst_params}')

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'dea53c5c-2b73-40a3-aea0-9423940f7440',
              'x': array([datetime.datetime(2023, 5, 22, 22, 40, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 22, 22, 41, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 22, 22, 42, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 5, 29, 22, 34, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 29, 22, 35, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 29, 22, 36, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1820.35327148, 1820.11865234, 1820.2421875 , .

In [17]:
pf.plot(column=best_params, show_titles=True, title=f'Portfolio {best_params}')

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e05b1ba6-c57e-4269-a162-b70252779b22',
              'x': array([datetime.datetime(2023, 5, 22, 22, 40, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 22, 22, 41, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 22, 22, 42, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 5, 29, 22, 34, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 29, 22, 35, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 29, 22, 36, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1820.35327148, 1820.11865234, 1820.2421875 , .